In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math as mt
import random as rng
import pickle
from sklearn import preprocessing

In [2]:
data = np.load("grouped.npy", allow_pickle=True)
target = np.load("groupedTarg.npy", allow_pickle=True)

In [3]:
data = data.tolist()
target = target.tolist()

val = []
valTarget = []

valAmount = mt.floor(len(target)/5)
trainAmount = len(target) - valAmount


for i in range(mt.floor(len(target)/5)):
    index = rng.randint(0, len(target)-1)
    val.append(data.pop(index))
    valTarget.append(target.pop(index))

train = data
trainTarget = target


In [4]:
def flatten(array):
    flat_array = []
    
    for group in array:
        for i in group:
            flat_array.append(i)
    return flat_array


trainFlat = flatten(train)
trainTargetFlat = flatten(trainTarget)
valFlat = flatten(val)
valTargetFlat = flatten(valTarget)

In [5]:
print(len(trainFlat[0]))
print(len(valFlat))

21
3016


In [6]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

scaledTrain = scaler.fit_transform(np.array(trainFlat))
scaledVal = scaler.transform(np.array(valFlat))

In [7]:
#PCA

from sklearn.decomposition import PCA
pca = PCA(.96)
scaledTrain = pca.fit_transform(scaledTrain)

scaledVal = pca.transform(scaledVal)

In [8]:
print(len(scaledVal[0]))

11


In [9]:
pickle.dump( scaler, open( "transforms/scaler.p", "wb" ) )
pickle.dump(pca, open("transforms/pca.p", "wb"))


In [10]:
def regroup(data, target, amount):
    grouped = []
    groupedTarg = []
    
    for i in range(amount):
        grouped.append([])
        groupedTarg.append([])
    
    total = 0
    for i in range(len(grouped)):
        j = total
        if i!= len(grouped)-1:
            while target[j] > target[j+1]:
                grouped[i].append(data[j])
                groupedTarg[i].append(target[j])
                j+=1
            grouped[i].append(data[j])
            groupedTarg[i].append(target[j])
            total = j + 1

    for i in range(total, len(target)):
        grouped[len(grouped)-1].append(data[i])
        groupedTarg[len(grouped)-1].append(target[i])
    
    return grouped, groupedTarg

In [11]:
train, targetTrain = regroup(scaledTrain.tolist(), trainTargetFlat, trainAmount)
val, valTargetTrain = regroup(scaledVal.tolist(), valTargetFlat, valAmount)



np.save("finished/stTrain", np.array(train))
np.save("finished/stTrainTarget", np.array(targetTrain))

np.save("finished/stVal", np.array(val))
np.save("finished/stValTarget", np.array(valTargetTrain))